In [1]:
# Import Modules
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
%matplotlib inline
import pyspark
import time
import lyricsgenius
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import billboard
import time
import requests
import tweepy
import re
import unicodedata
import GetOldTweets3 as got
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from langdetect import detect
from nltk.tokenize import word_tokenize
from string import punctuation

from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
from pyspark.ml.feature import CountVectorizer , IDF
from pyspark.mllib.linalg import Vector, Vectors
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.tree import RandomForest
from pyspark.mllib.evaluation import BinaryClassificationMetrics

import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')
from gensim import corpora, models
from pprint import pprint
from collections import Counter

APP_NAME = "Random Forest"
SPARK_URL = "local[*]"

spark = SparkSession.builder \
    .appName(APP_NAME) \
    .master(SPARK_URL) \
    .getOrCreate()

[nltk_data] Downloading package stopwords to /Users/pk/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/pk/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
%env SPOTIPY_CLIENT_ID=28aa32e2bfbc4a2183b24afc6e52a40c
%env SPOTIPY_CLIENT_SECRET=48c3512efc3f4098bdf80304e0a67bab

env: SPOTIPY_CLIENT_ID=28aa32e2bfbc4a2183b24afc6e52a40c
env: SPOTIPY_CLIENT_SECRET=48c3512efc3f4098bdf80304e0a67bab


In [3]:
sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials())

In [4]:
chart = billboard.ChartData('hot-100')

In [5]:
def strip_accents(text):

    try:
        text = unicode(text, 'utf-8')
    except NameError: # unicode is a default on python 3 
        pass

    text = unicodedata.normalize('NFD', text)\
           .encode('ascii', 'ignore')\
           .decode("utf-8")

    return str(text)

In [6]:
def regformat(text):
    transl_table = dict( [ (ord(x), ord(y)) for x,y in zip( u"‘’´“”–-",  u"'''\"\"--") ] )
    text = text.translate( transl_table )
    text = text.strip()
    newstring = ''
    for word in text.split():
        if word[0] != '(':
            newstring+=word.capitalize()+" "
        else:
            newstring+='('+word[1:].capitalize()+" "
    newstring = newstring.strip()
    return newstring

In [7]:
try:
    trackids = []
    for song in chart:
        title = song.title
        artist = song.artist
        artists = [regformat(x.strip()) for x in re.split(' Featuring | & | X |, | Duet With ', artist)]
        query = title
        if len(artists) > 1:
            for x in artists:
                query+=" "+x
        else:
            query+=" "+artist
        if sp.search(query)['tracks']['items'] != []: # Check if empty
            found = False
            for song in sp.search(query)['tracks']['items']:
                if not found:
                    if title in regformat(song['name']): # Check if the billboard title is in the song's title
                        foundartists = []
                        for singer in song['artists']:
                            foundartists.append(strip_accents(singer['name']))
                        result =  any(regformat(elem) in foundartists for elem in artists)
                        if result:
                            if song['id'] not in trackids:
                                trackids.append(song['id'])
                            found = True
                        elif regformat(foundartists[0]) == regformat(artist):
                            if song['id'] not in trackids:
                                trackids.append(song['id'])
                            found = True
                    elif regformat(song['name']) in title:
                        foundartists = []
                        for singer in song['artists']:
                            foundartists.append(strip_accents(singer['name']))
                        result =  any(regformat(elem) in foundartists for elem in artists)
                        if result:
                            if song['id'] not in trackids:
                                trackids.append(song['id'])
                            found = True
                        elif regformat(foundartists[0]) == regformat(artist):
                            if song['id'] not in trackids:
                                trackids.append(song['id'])
                            found = True
except:
    print('error')
print(len(trackids))
trackids

94


['24ySl2hOPGCDcxBxFIqWBu',
 '5v4GgrXPMghOnBBLmveLac',
 '7ytR5pFWmSjzHJIeQkgog4',
 '0VjIjW4GlUZAMYd2vXMi3b',
 '11VApNQCWLJdzxWrlmwzUa',
 '127QTOFJsJQp5LbJbu3A1y',
 '6WrI0LAC5M1Rw2MnX2ZvEg',
 '0nbXyq5TXYPCO7pr3N8S4I',
 '364dI1bYnvamSnBJ8JcNzN',
 '5yY9lUy8nbvjM1Uyo1Uqoc',
 '21jGcNKet2qwijlDFuPiPb',
 '1M4qEo4HE3PRaCOM7EXNJq',
 '3npzogdOEiVsieXmmhBepL',
 '4HBZA5flZLE435QTztThqH',
 '6wJYhPfqk3KGhHRG76WzOh',
 '39Yp9wwQiSRIDOvrVg7mbk',
 '5MwynWK9s4hlyKHqhkNn4A',
 '2Hc4t9N2nt37oO4sF9BynI',
 '0nGXi46VcQQ56ZJR428MKS',
 '4TnjEaWOeW0eKTKIEvJyCa',
 '4NhDYoQTYCdWHTvlbGVgwo',
 '07KXEDMj78x68D884wgVEm',
 '1Cv1YLb4q0RzL6pybtaMLo',
 '1jaTQ3nqY3oAAYyCTbIvnM',
 '7eJMfftS33KTjuF7lTsMCx',
 '5RqR4ZCCKJDcBLIn4sih9l',
 '5M8goiFYynmNxhueYW6grR',
 '0Al3wN3keprGTSgG8NBwjr',
 '7ce20yLkzuXXLUhzIDoZih',
 '3yOlyBJuViE2YSGn3nVE1K',
 '7szuecWAPwGoV1e5vGu8tl',
 '1z6wl2rwKatfmza6usanWw',
 '1raaNykBg1bDnWENUiglUA',
 '3ZCTVFBt2Brf31RLEnCkWJ',
 '2b8fOow8UzyDFAE27YhOZM',
 '10xuH52YSXHWFwCaF1fsDd',
 '3nS9a01VvXHQriLqJYwRqG',
 

In [8]:
tracks = []

for track in trackids:
    Week = dt.datetime.now().strftime('%m/%d/%Y')
    Song = None
    Peak = None
    Performer = []
    SongID = None
    WeeksOnChart = None
    spotify_track_explicit = None
    spotify_track_duration_ms = None
    spotify_track_popularity = None
    danceability = None
    energy = None
    key = None
    loudness = None
    mode = None
    speechiness = None
    acousticness = None
    instrumentalness = None
    liveness = None
    valence = None
    tempo = None
    time_signature = None
    debut = None
    for k, v in sp.track(track).items():
        if k == 'artists':
            for artist in v:
                for key, val in artist.items():
                    if key == 'name':
                        Performer.append(val)
        if k == 'duration_ms':
            spotify_track_duration_ms = v
        if k == 'explicit':
            spotify_track_explicit = v
        if k == 'popularity':
            spotify_track_popularity = v
        if k == 'name':
            Song = re.split("(\(feat.*\))|(\(with.*\))", v)[0].strip()
    if sp.audio_features(track)[0]:
        for k, v in sp.audio_features(track)[0].items():
            if k == 'danceability':
                danceability = v
            if k == 'energy':
                energy = v
            if k == 'key':
                key = v
            if k == 'loudness':
                loudness = v
            if k == 'mode':
                mode = v
            if k == 'speechiness':
                speechiness = v
            if k == 'acousticness':
                acousticness = v
            if k == 'instrumentalness':
                instrumentalness = v
            if k == 'liveness':
                liveness = v
            if k == 'valence':
                valence = v
            if k == 'tempo':
                tempo = v
            if k == 'time_signature':
                time_signature = v
    for song in chart:
        if (song.title in regformat(Song) and Performer[0] in song.artist):
            Performer[0] = song.artist
            # Only gets songs that are currently at their peak
            if song.peakPos: #Checks if peak position exists
                if song.rank == song.peakPos: # Checks if current rank is equal to peak position
                    Peak = song.rank
                else:
                    Peak = 'N/A'
            else: 
                Peak = song.rank # Sets song's current rank as peak position if peak position does not exist
            if song.lastPos: # Checks if last position exists
                if (song.lastPos != song.peakPos and song.lastPos > song.rank): # Checks if last position is not equal to peak position and that the song's last position was lower than its current rank
                    debut = song.lastPos
                else:
                    debut = 'N/A'
            else:
                debut = song.rank # Sets song's current rank as debut if last position does not exist
            WeeksOnChart = song.weeks
            
    if debut != 'N/A' and Peak != 'N/A':
        SongID = Song+Performer[0]
        tracks.append([Week, SongID, Song, Performer[0], Peak, WeeksOnChart, spotify_track_explicit, spotify_track_duration_ms, spotify_track_popularity, \
                      danceability, energy, key, loudness, mode, speechiness, acousticness, instrumentalness, liveness, valence, tempo, time_signature, debut])
    
tracks    

[['06/04/2020',
  'Rain On MeLady Gaga & Ariana Grande',
  'Rain On Me',
  'Lady Gaga & Ariana Grande',
  1,
  1,
  False,
  182200,
  94,
  0.672,
  0.855,
  9,
  -3.764,
  1,
  0.0397,
  0.021,
  0,
  0.323,
  0.646,
  123.056,
  4,
  1],
 ['06/04/2020',
  'ROCKSTARDaBaby Featuring Roddy Ricch',
  'ROCKSTAR',
  'DaBaby Featuring Roddy Ricch',
  3,
  6,
  True,
  181733,
  98,
  0.746,
  0.69,
  11,
  -7.956,
  1,
  0.164,
  0.247,
  0,
  0.101,
  0.497,
  89.977,
  4,
  4],
 ['06/04/2020',
  'Blueberry FaygoLil Mosey',
  'Blueberry Faygo',
  'Lil Mosey',
  16,
  16,
  True,
  162546,
  93,
  0.774,
  0.554,
  0,
  -7.909,
  1,
  0.0383,
  0.207,
  0,
  0.132,
  0.349,
  99.034,
  4,
  17],
 ['06/04/2020',
  'Does To MeLuke Combs Featuring Eric Church',
  'Does To Me',
  'Luke Combs Featuring Eric Church',
  20,
  11,
  False,
  223320,
  77,
  0.616,
  0.849,
  0,
  -4.897,
  1,
  0.0373,
  0.311,
  0,
  0.311,
  0.638,
  113.065,
  4,
  22],
 ['06/04/2020',
  'Sunday BestSurfaces',


In [9]:
newtracks = pd.DataFrame(tracks, columns =['Week', 'SongID', 'Song', 'Performer', 'Peak', 'WeeksOnChart', 'spotify_track_explicit', 'spotify_track_duration_ms', 'spotify_track_popularity', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature', 'debut']) 



In [10]:
newtracks

,Week,SongID,Song,Performer,Peak,WeeksOnChart,spotify_track_explicit,spotify_track_duration_ms,spotify_track_popularity,danceability,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,debut
0,06/04/2020,Rain On MeLady Gaga & Ariana Grande,Rain On Me,Lady Gaga & Ariana Grande,1.0,1.0,False,182200,94,0.672,...,-3.764,1,0.0397,0.02100,0.000000,0.3230,0.646,123.056,4,1.0
1,06/04/2020,ROCKSTARDaBaby Featuring Roddy Ricch,ROCKSTAR,DaBaby Featuring Roddy Ricch,3.0,6.0,True,181733,98,0.746,...,-7.956,1,0.1640,0.24700,0.000000,0.1010,0.497,89.977,4,4.0
2,06/04/2020,Blueberry FaygoLil Mosey,Blueberry Faygo,Lil Mosey,16.0,16.0,True,162546,93,0.774,...,-7.909,1,0.0383,0.20700,0.000000,0.1320,0.349,99.034,4,17.0
3,06/04/2020,Does To MeLuke Combs Featuring Eric Church,Does To Me,Luke Combs Featuring Eric Church,20.0,11.0,False,223320,77,0.616,...,-4.897,1,0.0373,0.31100,0.000000,0.3110,0.638,113.065,4,22.0
4,06/04/2020,Sunday BestSurfaces,Sunday Best,Surfaces,24.0,13.0,False,158571,94,0.878,...,-6.832,1,0.0578,0.18300,0.000000,0.0714,0.694,112.022,4,26.0
5,06/04/2020,WHATS POPPINJack Harlow,WHATS POPPIN,Jack Harlow,25.0,16.0,True,139741,92,0.923,...,-6.671,0,0.2450,0.01700,0.000000,0.2720,0.826,145.062,4,28.0
6,06/04/2020,death bed (coffee for your head)Powfu Featurin...,death bed (coffee for your head),Powfu Featuring beabadoobee,26.0,11.0,False,173333,98,0.726,...,-8.765,0,0.1350,0.73100,0.000000,0.6960,0.348,144.026,4,29.0
7,06/04/2020,Party GirlStaySolidRocky,Party Girl,StaySolidRocky,27.0,5.0,False,147800,91,0.728,...,-9.966,0,0.0622,0.74900,0.000000,0.0996,0.629,130.022,4,38.0
8,06/04/2020,I Hope You’re Happy NowCarly Pearce & Lee Brice,I Hope You’re Happy Now,Carly Pearce & Lee Brice,29.0,21.0,False,198688,77,0.591,...,-4.725,1,0.0366,0.16800,0.000009,0.1420,0.306,118.024,4,35.0
9,06/04/2020,Before You GoLewis Capaldi,Before You Go,Lewis Capaldi,30.0,17.0,False,215106,88,0.459,...,-4.858,1,0.0573,0.60400,0.000000,0.0885,0.183,111.881,4,36.0


In [11]:
newtracks["Lyrics"] = 0

In [12]:
genius = lyricsgenius.Genius("lRSmSx2mp4ZU4ZovVyII_bDmIN2ROTR_0yU9HDCZI47VKNkZw-sBf5b4pruTYMam")
genius.remove_section_headers = True
genius.verbose = False
genius.skip_non_songs = True

In [13]:
for idx, values in newtracks.iterrows():
    try:
        lyrics = genius.search_song(values['Song'],values['Performer']).lyrics
        newtracks["Lyrics"][idx] = lyrics
    except:
        print("An exception occurred")

Timeout raised and caught:
HTTPSConnectionPool(host='api.genius.com', port=443): Read timed out. (read timeout=5)
An exception occurred


/Users/pk/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Users/pk/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [14]:
newtracks.drop(newtracks[newtracks['Lyrics'] == 0].index, inplace = True)

In [15]:
newtracks["LyricPositive"] = 0
newtracks["LyricNeutral"] = 0
newtracks["LyricNegative"] = 0

In [16]:
sia = SentimentIntensityAnalyzer()
for idx, values in newtracks.iterrows():
    if detect(values['Lyrics']) == 'en':
        num_positive = 0
        num_neutral = 0
        num_negative = 0
        for line in values['Lyrics'].splitlines():
            if line != "":
                comp = sia.polarity_scores(line)   
                comp = comp['compound']
                if comp >= 0.5:
                    num_positive += 1
                elif comp > -0.5 and comp < 0.5:
                    num_neutral += 1
                else:
                    num_negative += 1
        num_total = num_negative + num_neutral + num_positive
        percent_negative = (num_negative/float(num_total))*100
        percent_neutral = (num_neutral/float(num_total))*100
        percent_positive = (num_positive/float(num_total))*100
        newtracks['LyricPositive'][idx] = percent_positive
        newtracks['LyricNeutral'][idx] = percent_neutral
        newtracks['LyricNegative'][idx] = percent_negative
newtracks

/Users/pk/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/pk/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/pk/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Week,SongID,Song,Performer,Peak,WeeksOnChart,spotify_track_explicit,spotify_track_duration_ms,spotify_track_popularity,danceability,...,instrumentalness,liveness,valence,tempo,time_signature,debut,Lyrics,LyricPositive,LyricNeutral,LyricNegative
1,06/04/2020,ROCKSTARDaBaby Featuring Roddy Ricch,ROCKSTAR,DaBaby Featuring Roddy Ricch,3.0,6.0,True,181733,98,0.746,...,0.000000,0.1010,0.497,89.977,4,4.0,"Woo, woo\nI pull up like\nHow you pull up, Bab...",15,73,10
2,06/04/2020,Blueberry FaygoLil Mosey,Blueberry Faygo,Lil Mosey,16.0,16.0,True,162546,93,0.774,...,0.000000,0.1320,0.349,99.034,4,17.0,"Damn, Callan (Damn, Callan)\n\nOne bad bitch a...",1,61,36
3,06/04/2020,Does To MeLuke Combs Featuring Eric Church,Does To Me,Luke Combs Featuring Eric Church,20.0,11.0,False,223320,77,0.616,...,0.000000,0.3110,0.638,113.065,4,22.0,I was a third-string dreamer on a second-place...,3,92,3
4,06/04/2020,Sunday BestSurfaces,Sunday Best,Surfaces,24.0,13.0,False,158571,94,0.878,...,0.000000,0.0714,0.694,112.022,4,26.0,"Good, feeling good\n\nAyy, feeling good, like ...",42,57,0
5,06/04/2020,WHATS POPPINJack Harlow,WHATS POPPIN,Jack Harlow,25.0,16.0,True,139741,92,0.923,...,0.000000,0.2720,0.826,145.062,4,28.0,"What's poppin'? (Pooh, you a fool for this one...",7,84,7
6,06/04/2020,death bed (coffee for your head)Powfu Featurin...,death bed (coffee for your head),Powfu Featuring beabadoobee,26.0,11.0,False,173333,98,0.726,...,0.000000,0.6960,0.348,144.026,4,29.0,"Don't stay awake for too long, don't go to bed...",11,86,2
7,06/04/2020,Party GirlStaySolidRocky,Party Girl,StaySolidRocky,27.0,5.0,False,147800,91,0.728,...,0.000000,0.0996,0.629,130.022,4,38.0,"Girls just wanna have fun\nSo they party, so t...",45,43,10
8,06/04/2020,I Hope You’re Happy NowCarly Pearce & Lee Brice,I Hope You’re Happy Now,Carly Pearce & Lee Brice,29.0,21.0,False,198688,77,0.591,...,0.000009,0.1420,0.306,118.024,4,35.0,"It's all on me, it's my mistake\nI said ""I don...",25,65,9
9,06/04/2020,Before You GoLewis Capaldi,Before You Go,Lewis Capaldi,30.0,17.0,False,215106,88,0.459,...,0.000000,0.0885,0.183,111.881,4,36.0,I fell by the wayside like everyone else\nI ha...,0,80,20
10,06/04/2020,After A FewTravis Denning,After A Few,Travis Denning,33.0,9.0,False,213493,74,0.617,...,0.000000,0.1940,0.695,106.998,4,37.0,I probably shouldn't have said hello\nWhen I s...,6,94,0


In [17]:
newtracks['LyricSentimentTotal'] = newtracks.apply(lambda row: row.LyricPositive + row.LyricNeutral + row.LyricNegative, axis=1)
newtracks.drop(newtracks[newtracks['LyricSentimentTotal'] == 0].index, inplace = True)
newtracks = newtracks.drop(columns=['LyricSentimentTotal'])
newtracks

,Week,SongID,Song,Performer,Peak,WeeksOnChart,spotify_track_explicit,spotify_track_duration_ms,spotify_track_popularity,danceability,...,instrumentalness,liveness,valence,tempo,time_signature,debut,Lyrics,LyricPositive,LyricNeutral,LyricNegative
1,06/04/2020,ROCKSTARDaBaby Featuring Roddy Ricch,ROCKSTAR,DaBaby Featuring Roddy Ricch,3.0,6.0,True,181733,98,0.746,...,0.000000,0.1010,0.497,89.977,4,4.0,"Woo, woo\nI pull up like\nHow you pull up, Bab...",15,73,10
2,06/04/2020,Blueberry FaygoLil Mosey,Blueberry Faygo,Lil Mosey,16.0,16.0,True,162546,93,0.774,...,0.000000,0.1320,0.349,99.034,4,17.0,"Damn, Callan (Damn, Callan)\n\nOne bad bitch a...",1,61,36
3,06/04/2020,Does To MeLuke Combs Featuring Eric Church,Does To Me,Luke Combs Featuring Eric Church,20.0,11.0,False,223320,77,0.616,...,0.000000,0.3110,0.638,113.065,4,22.0,I was a third-string dreamer on a second-place...,3,92,3
4,06/04/2020,Sunday BestSurfaces,Sunday Best,Surfaces,24.0,13.0,False,158571,94,0.878,...,0.000000,0.0714,0.694,112.022,4,26.0,"Good, feeling good\n\nAyy, feeling good, like ...",42,57,0
5,06/04/2020,WHATS POPPINJack Harlow,WHATS POPPIN,Jack Harlow,25.0,16.0,True,139741,92,0.923,...,0.000000,0.2720,0.826,145.062,4,28.0,"What's poppin'? (Pooh, you a fool for this one...",7,84,7
6,06/04/2020,death bed (coffee for your head)Powfu Featurin...,death bed (coffee for your head),Powfu Featuring beabadoobee,26.0,11.0,False,173333,98,0.726,...,0.000000,0.6960,0.348,144.026,4,29.0,"Don't stay awake for too long, don't go to bed...",11,86,2
7,06/04/2020,Party GirlStaySolidRocky,Party Girl,StaySolidRocky,27.0,5.0,False,147800,91,0.728,...,0.000000,0.0996,0.629,130.022,4,38.0,"Girls just wanna have fun\nSo they party, so t...",45,43,10
8,06/04/2020,I Hope You’re Happy NowCarly Pearce & Lee Brice,I Hope You’re Happy Now,Carly Pearce & Lee Brice,29.0,21.0,False,198688,77,0.591,...,0.000009,0.1420,0.306,118.024,4,35.0,"It's all on me, it's my mistake\nI said ""I don...",25,65,9
9,06/04/2020,Before You GoLewis Capaldi,Before You Go,Lewis Capaldi,30.0,17.0,False,215106,88,0.459,...,0.000000,0.0885,0.183,111.881,4,36.0,I fell by the wayside like everyone else\nI ha...,0,80,20
10,06/04/2020,After A FewTravis Denning,After A Few,Travis Denning,33.0,9.0,False,213493,74,0.617,...,0.000000,0.1940,0.695,106.998,4,37.0,I probably shouldn't have said hello\nWhen I s...,6,94,0


In [18]:
# Creates New Twitter Dataset with Sentiment Fields
twitter = newtracks[['Week', 'Song', 'Performer', 'SongID']]
twitter['Tweets'] = 0
twitter['TweetPositive'] = 0
twitter['TweetNeutral'] = 0
twitter['TweetNegative'] = 0
twitter

/Users/pk/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/pk/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Users/pk/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexe

,Week,Song,Performer,SongID,Tweets,TweetPositive,TweetNeutral,TweetNegative
1,06/04/2020,ROCKSTAR,DaBaby Featuring Roddy Ricch,ROCKSTARDaBaby Featuring Roddy Ricch,0,0,0,0
2,06/04/2020,Blueberry Faygo,Lil Mosey,Blueberry FaygoLil Mosey,0,0,0,0
3,06/04/2020,Does To Me,Luke Combs Featuring Eric Church,Does To MeLuke Combs Featuring Eric Church,0,0,0,0
4,06/04/2020,Sunday Best,Surfaces,Sunday BestSurfaces,0,0,0,0
5,06/04/2020,WHATS POPPIN,Jack Harlow,WHATS POPPINJack Harlow,0,0,0,0
6,06/04/2020,death bed (coffee for your head),Powfu Featuring beabadoobee,death bed (coffee for your head)Powfu Featurin...,0,0,0,0
7,06/04/2020,Party Girl,StaySolidRocky,Party GirlStaySolidRocky,0,0,0,0
8,06/04/2020,I Hope You’re Happy Now,Carly Pearce & Lee Brice,I Hope You’re Happy NowCarly Pearce & Lee Brice,0,0,0,0
9,06/04/2020,Before You Go,Lewis Capaldi,Before You GoLewis Capaldi,0,0,0,0
10,06/04/2020,After A Few,Travis Denning,After A FewTravis Denning,0,0,0,0


In [19]:
# Twitter Data Collection
iteration = 0
for idx, values in twitter.iterrows():
    if iteration == 15:
        time.sleep(500) # Makes the operation sleep for 8 minutes as to not overload API with a large number of requests
        iteration = 0
    tweets = []
    d = dt.timedelta(days=14)
    enddate = dt.datetime.strptime(values['Week'], "%m/%d/%Y")
    startdate = enddate-d
    tweetCriteria = got.manager.TweetCriteria().setQuerySearch(values['Song'] + " " + values['Performer']).setSince(startdate.strftime("%Y-%m-%d")).setUntil(enddate.strftime("%Y-%m-%d")).setMaxTweets(100)
    tweet = got.manager.TweetManager.getTweets(tweetCriteria)
    stopwords12 = set(stopwords.words('english') + list(punctuation))
    for x in tweet:
        x = x.text
        x = x.lower() # convert text to lower-case
        x = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', '', x) # remove URLs
        x = re.sub('@[^\s]+', '', x) # remove usernames
        x = re.sub(r'#([^\s]+)', r'\1', x) # remove the # in #hashtag
        try:
            # Checks if the tweets are in english for sentiment analysis
            if detect(x) == 'en':
                tweets.append(x)
        except:
            print("LangDetect Error")
    twitter['Tweets'][idx] = tweets
    iteration+=1

/Users/pk/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/pk/anaconda3/lib/python3.7/site-packages/pandas/core/series.py:915: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.loc[key] = value
/Users/pk/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3267: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


LangDetect Error
LangDetect Error
LangDetect Error
LangDetect Error
LangDetect Error
LangDetect Error


In [20]:
sia = SentimentIntensityAnalyzer()
for idx, values in twitter.iterrows():
    newtweets = []
    song = values['Song'].lower()
    performer = values['Performer'].lower()
    for x in values['Tweets']:
        text = x.replace(song, "").replace(performer, "").replace('-','').strip()
        if text:
            newtweets.append(text)
    twitter['Tweets'][idx] = newtweets
    num_positive = 0
    num_neutral = 0
    num_negative = 0
    for x in newtweets:
        if x != "":
            comp = sia.polarity_scores(x)   
            comp = comp['compound']
            if comp >= 0.5:
                num_positive += 1
            elif comp > -0.5 and comp < 0.5:
                num_neutral += 1
            else:
                num_negative += 1
    num_total = num_negative + num_neutral + num_positive
    if num_total != 0:
        percent_negative = (num_negative/float(num_total))*100
        percent_neutral = (num_neutral/float(num_total))*100
        percent_positive = (num_positive/float(num_total))*100
        twitter['TweetPositive'][idx] = percent_positive
        twitter['TweetNeutral'][idx] = percent_neutral
        twitter['TweetNegative'][idx] = percent_negative

/Users/pk/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
/Users/pk/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/pk/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/pk/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:31: SettingW

In [21]:
twitter['TweetSentimentTotal'] = twitter.apply(lambda row: row.TweetPositive + row.TweetNeutral + row.TweetNegative, axis=1)

/Users/pk/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [22]:
twitter

,Week,Song,Performer,SongID,Tweets,TweetPositive,TweetNeutral,TweetNegative,TweetSentimentTotal
1,06/04/2020,ROCKSTAR,DaBaby Featuring Roddy Ricch,ROCKSTARDaBaby Featuring Roddy Ricch,[music news on the aria singles chart dababy’s...,11,76,11,98
2,06/04/2020,Blueberry Faygo,Lil Mosey,Blueberry FaygoLil Mosey,[has climbed 15 places from 100th place to 85t...,4,95,0,99
3,06/04/2020,Does To Me,Luke Combs Featuring Eric Church,Does To MeLuke Combs Featuring Eric Church,"[and that might not mean much to you by, now ...",0,100,0,100
4,06/04/2020,Sunday Best,Surfaces,Sunday BestSurfaces,"[by, internacionais: boss bitchdoja cat genius...",12,85,2,99
5,06/04/2020,WHATS POPPIN,Jack Harlow,WHATS POPPINJack Harlow,[sub subscribe now to dallas worldstarnews you...,5,94,0,99
6,06/04/2020,death bed (coffee for your head),Powfu Featuring beabadoobee,death bed (coffee for your head)Powfu Featurin...,[tmn weekly top 100 highlights week ending: 05...,0,100,0,100
7,06/04/2020,Party Girl,StaySolidRocky,Party GirlStaySolidRocky,[there's a second debut on the bottom end: '' ...,7,91,1,99
8,06/04/2020,I Hope You’re Happy Now,Carly Pearce & Lee Brice,I Hope You’re Happy NowCarly Pearce & Lee Brice,[carly pearce lee brice i hope you're happy n...,78,20,1,99
9,06/04/2020,Before You Go,Lewis Capaldi,Before You GoLewis Capaldi,"[by but it’s newtmas., what do i have to do t...",15,81,3,99
10,06/04/2020,After A Few,Travis Denning,After A FewTravis Denning,[rides country hit ‘’ to no. 1 on emerging art...,15,81,2,98


In [23]:
newtracks = pd.merge(newtracks, twitter, how='inner', on=['SongID'])
newtracks

,Week_x,SongID,Song_x,Performer_x,Peak,WeeksOnChart,spotify_track_explicit,spotify_track_duration_ms,spotify_track_popularity,danceability,...,LyricNeutral,LyricNegative,Week_y,Song_y,Performer_y,Tweets,TweetPositive,TweetNeutral,TweetNegative,TweetSentimentTotal
0,06/04/2020,ROCKSTARDaBaby Featuring Roddy Ricch,ROCKSTAR,DaBaby Featuring Roddy Ricch,3.0,6.0,True,181733,98,0.746,...,73,10,06/04/2020,ROCKSTAR,DaBaby Featuring Roddy Ricch,[music news on the aria singles chart dababy’s...,11,76,11,98
1,06/04/2020,Blueberry FaygoLil Mosey,Blueberry Faygo,Lil Mosey,16.0,16.0,True,162546,93,0.774,...,61,36,06/04/2020,Blueberry Faygo,Lil Mosey,[has climbed 15 places from 100th place to 85t...,4,95,0,99
2,06/04/2020,Does To MeLuke Combs Featuring Eric Church,Does To Me,Luke Combs Featuring Eric Church,20.0,11.0,False,223320,77,0.616,...,92,3,06/04/2020,Does To Me,Luke Combs Featuring Eric Church,"[and that might not mean much to you by, now ...",0,100,0,100
3,06/04/2020,Sunday BestSurfaces,Sunday Best,Surfaces,24.0,13.0,False,158571,94,0.878,...,57,0,06/04/2020,Sunday Best,Surfaces,"[by, internacionais: boss bitchdoja cat genius...",12,85,2,99
4,06/04/2020,WHATS POPPINJack Harlow,WHATS POPPIN,Jack Harlow,25.0,16.0,True,139741,92,0.923,...,84,7,06/04/2020,WHATS POPPIN,Jack Harlow,[sub subscribe now to dallas worldstarnews you...,5,94,0,99
5,06/04/2020,death bed (coffee for your head)Powfu Featurin...,death bed (coffee for your head),Powfu Featuring beabadoobee,26.0,11.0,False,173333,98,0.726,...,86,2,06/04/2020,death bed (coffee for your head),Powfu Featuring beabadoobee,[tmn weekly top 100 highlights week ending: 05...,0,100,0,100
6,06/04/2020,Party GirlStaySolidRocky,Party Girl,StaySolidRocky,27.0,5.0,False,147800,91,0.728,...,43,10,06/04/2020,Party Girl,StaySolidRocky,[there's a second debut on the bottom end: '' ...,7,91,1,99
7,06/04/2020,I Hope You’re Happy NowCarly Pearce & Lee Brice,I Hope You’re Happy Now,Carly Pearce & Lee Brice,29.0,21.0,False,198688,77,0.591,...,65,9,06/04/2020,I Hope You’re Happy Now,Carly Pearce & Lee Brice,[carly pearce lee brice i hope you're happy n...,78,20,1,99
8,06/04/2020,Before You GoLewis Capaldi,Before You Go,Lewis Capaldi,30.0,17.0,False,215106,88,0.459,...,80,20,06/04/2020,Before You Go,Lewis Capaldi,"[by but it’s newtmas., what do i have to do t...",15,81,3,99
9,06/04/2020,After A FewTravis Denning,After A Few,Travis Denning,33.0,9.0,False,213493,74,0.617,...,94,0,06/04/2020,After A Few,Travis Denning,[rides country hit ‘’ to no. 1 on emerging art...,15,81,2,98


In [24]:
newtracks = newtracks.drop(columns=['Week_y', 'Song_y', 'Performer_y'])
newtracks = newtracks.rename(columns={"Week_x": "Week", "Song_x": "Song", "Performer_x": "Performer"})
newtracks.drop(newtracks[newtracks['TweetSentimentTotal'] == 0].index, inplace = True)
newtracks = newtracks.drop(columns=['TweetSentimentTotal'])
newtracks


,Week,SongID,Song,Performer,Peak,WeeksOnChart,spotify_track_explicit,spotify_track_duration_ms,spotify_track_popularity,danceability,...,time_signature,debut,Lyrics,LyricPositive,LyricNeutral,LyricNegative,Tweets,TweetPositive,TweetNeutral,TweetNegative
0,06/04/2020,ROCKSTARDaBaby Featuring Roddy Ricch,ROCKSTAR,DaBaby Featuring Roddy Ricch,3.0,6.0,True,181733,98,0.746,...,4,4.0,"Woo, woo\nI pull up like\nHow you pull up, Bab...",15,73,10,[music news on the aria singles chart dababy’s...,11,76,11
1,06/04/2020,Blueberry FaygoLil Mosey,Blueberry Faygo,Lil Mosey,16.0,16.0,True,162546,93,0.774,...,4,17.0,"Damn, Callan (Damn, Callan)\n\nOne bad bitch a...",1,61,36,[has climbed 15 places from 100th place to 85t...,4,95,0
2,06/04/2020,Does To MeLuke Combs Featuring Eric Church,Does To Me,Luke Combs Featuring Eric Church,20.0,11.0,False,223320,77,0.616,...,4,22.0,I was a third-string dreamer on a second-place...,3,92,3,"[and that might not mean much to you by, now ...",0,100,0
3,06/04/2020,Sunday BestSurfaces,Sunday Best,Surfaces,24.0,13.0,False,158571,94,0.878,...,4,26.0,"Good, feeling good\n\nAyy, feeling good, like ...",42,57,0,"[by, internacionais: boss bitchdoja cat genius...",12,85,2
4,06/04/2020,WHATS POPPINJack Harlow,WHATS POPPIN,Jack Harlow,25.0,16.0,True,139741,92,0.923,...,4,28.0,"What's poppin'? (Pooh, you a fool for this one...",7,84,7,[sub subscribe now to dallas worldstarnews you...,5,94,0
5,06/04/2020,death bed (coffee for your head)Powfu Featurin...,death bed (coffee for your head),Powfu Featuring beabadoobee,26.0,11.0,False,173333,98,0.726,...,4,29.0,"Don't stay awake for too long, don't go to bed...",11,86,2,[tmn weekly top 100 highlights week ending: 05...,0,100,0
6,06/04/2020,Party GirlStaySolidRocky,Party Girl,StaySolidRocky,27.0,5.0,False,147800,91,0.728,...,4,38.0,"Girls just wanna have fun\nSo they party, so t...",45,43,10,[there's a second debut on the bottom end: '' ...,7,91,1
7,06/04/2020,I Hope You’re Happy NowCarly Pearce & Lee Brice,I Hope You’re Happy Now,Carly Pearce & Lee Brice,29.0,21.0,False,198688,77,0.591,...,4,35.0,"It's all on me, it's my mistake\nI said ""I don...",25,65,9,[carly pearce lee brice i hope you're happy n...,78,20,1
8,06/04/2020,Before You GoLewis Capaldi,Before You Go,Lewis Capaldi,30.0,17.0,False,215106,88,0.459,...,4,36.0,I fell by the wayside like everyone else\nI ha...,0,80,20,"[by but it’s newtmas., what do i have to do t...",15,81,3
9,06/04/2020,After A FewTravis Denning,After A Few,Travis Denning,33.0,9.0,False,213493,74,0.617,...,4,37.0,I probably shouldn't have said hello\nWhen I s...,6,94,0,[rides country hit ‘’ to no. 1 on emerging art...,15,81,2


In [25]:
newtracks = newtracks.reset_index()
newtracks = newtracks.drop(columns=['index'])

In [26]:
newtracks

,Week,SongID,Song,Performer,Peak,WeeksOnChart,spotify_track_explicit,spotify_track_duration_ms,spotify_track_popularity,danceability,...,time_signature,debut,Lyrics,LyricPositive,LyricNeutral,LyricNegative,Tweets,TweetPositive,TweetNeutral,TweetNegative
0,06/04/2020,ROCKSTARDaBaby Featuring Roddy Ricch,ROCKSTAR,DaBaby Featuring Roddy Ricch,3.0,6.0,True,181733,98,0.746,...,4,4.0,"Woo, woo\nI pull up like\nHow you pull up, Bab...",15,73,10,[music news on the aria singles chart dababy’s...,11,76,11
1,06/04/2020,Blueberry FaygoLil Mosey,Blueberry Faygo,Lil Mosey,16.0,16.0,True,162546,93,0.774,...,4,17.0,"Damn, Callan (Damn, Callan)\n\nOne bad bitch a...",1,61,36,[has climbed 15 places from 100th place to 85t...,4,95,0
2,06/04/2020,Does To MeLuke Combs Featuring Eric Church,Does To Me,Luke Combs Featuring Eric Church,20.0,11.0,False,223320,77,0.616,...,4,22.0,I was a third-string dreamer on a second-place...,3,92,3,"[and that might not mean much to you by, now ...",0,100,0
3,06/04/2020,Sunday BestSurfaces,Sunday Best,Surfaces,24.0,13.0,False,158571,94,0.878,...,4,26.0,"Good, feeling good\n\nAyy, feeling good, like ...",42,57,0,"[by, internacionais: boss bitchdoja cat genius...",12,85,2
4,06/04/2020,WHATS POPPINJack Harlow,WHATS POPPIN,Jack Harlow,25.0,16.0,True,139741,92,0.923,...,4,28.0,"What's poppin'? (Pooh, you a fool for this one...",7,84,7,[sub subscribe now to dallas worldstarnews you...,5,94,0
5,06/04/2020,death bed (coffee for your head)Powfu Featurin...,death bed (coffee for your head),Powfu Featuring beabadoobee,26.0,11.0,False,173333,98,0.726,...,4,29.0,"Don't stay awake for too long, don't go to bed...",11,86,2,[tmn weekly top 100 highlights week ending: 05...,0,100,0
6,06/04/2020,Party GirlStaySolidRocky,Party Girl,StaySolidRocky,27.0,5.0,False,147800,91,0.728,...,4,38.0,"Girls just wanna have fun\nSo they party, so t...",45,43,10,[there's a second debut on the bottom end: '' ...,7,91,1
7,06/04/2020,I Hope You’re Happy NowCarly Pearce & Lee Brice,I Hope You’re Happy Now,Carly Pearce & Lee Brice,29.0,21.0,False,198688,77,0.591,...,4,35.0,"It's all on me, it's my mistake\nI said ""I don...",25,65,9,[carly pearce lee brice i hope you're happy n...,78,20,1
8,06/04/2020,Before You GoLewis Capaldi,Before You Go,Lewis Capaldi,30.0,17.0,False,215106,88,0.459,...,4,36.0,I fell by the wayside like everyone else\nI ha...,0,80,20,"[by but it’s newtmas., what do i have to do t...",15,81,3
9,06/04/2020,After A FewTravis Denning,After A Few,Travis Denning,33.0,9.0,False,213493,74,0.617,...,4,37.0,I probably shouldn't have said hello\nWhen I s...,6,94,0,[rides country hit ‘’ to no. 1 on emerging art...,15,81,2


In [27]:
topics = pd.read_csv("Data/LDA.csv")

In [28]:
def lemmatize_stemming(text):
    stemmer = PorterStemmer()
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

In [29]:
words = []
for lyrics in topics['Lyrics']:
    for word in gensim.utils.simple_preprocess(lyrics):
        if word not in gensim.parsing.preprocessing.STOPWORDS and len(word) > 3:
            words.append(lemmatize_stemming(word))
add_stop_words = [word for word, count in Counter(words).most_common() if count > 750]

In [30]:
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            token = lemmatize_stemming(token)
            if token not in add_stop_words:
                result.append(token)
    return result

In [31]:
processed_docs = topics['Lyrics'].map(preprocess)

In [32]:
dictionary = gensim.corpora.Dictionary(processed_docs)
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [33]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

In [34]:
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]

In [35]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)
newtracks['Topic 0'] = 0.0
newtracks['Topic 1'] = 0.0
newtracks['Topic 2'] = 0.0
newtracks['Topic 3'] = 0.0
newtracks['Topic 4'] = 0.0
newtracks['Topic 5'] = 0.0
newtracks['Topic 6'] = 0.0
newtracks['Topic 7'] = 0.0
newtracks['Topic 8'] = 0.0
newtracks['Topic 9'] = 0.0

In [36]:
for idx, values in newtracks.iterrows():
    bow_vector = dictionary.doc2bow(preprocess(values['Lyrics']))
    for indx, score in sorted(lda_model_tfidf[bow_vector], key=lambda tup: -1*tup[1]):
        newtracks['Topic ' + str(indx)][idx] = score

/Users/pk/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [37]:
featureset = newtracks
featureset.dtypes

Week                          object
SongID                        object
Song                          object
Performer                     object
Peak                         float64
WeeksOnChart                 float64
spotify_track_explicit          bool
spotify_track_duration_ms      int64
spotify_track_popularity       int64
danceability                 float64
energy                       float64
key                            int64
loudness                     float64
mode                           int64
speechiness                  float64
acousticness                 float64
instrumentalness             float64
liveness                     float64
valence                      float64
tempo                        float64
time_signature                 int64
debut                        float64
Lyrics                        object
LyricPositive                  int64
LyricNeutral                   int64
LyricNegative                  int64
Tweets                        object
T

In [38]:
featureset = featureset.drop(columns=['Week', 'SongID', 'Song','Performer', 'Tweets', 'Lyrics'])

In [39]:
featureset['Classification'] = -1

In [40]:
for idx, values in featureset.iterrows():
    number = -1
    if values['Peak'] == 1:
        number = 0
    elif values['Peak'] < 6:
        number = 1
    elif values['Peak'] < 16:
        number = 2
    elif values['Peak'] < 31:
        number = 3
    elif values['Peak'] < 51:
        number = 4
    elif values['Peak'] < 76:
        number = 5
    elif values['Peak'] < 101:
        number = 6
    else:
        number = -1
    featureset['Classification'][idx] = number

/Users/pk/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [41]:
featureset = featureset.drop(columns=['Peak'])

In [42]:
for idx, values in featureset.iterrows():
    if isinstance(values['key'], int) == False:
        featureset['key'][idx] = -1

In [43]:
featureset['key']

0     11
1      0
2      0
3      5
4     11
5      8
6      6
7     10
8      3
9     10
10     4
11     0
12    11
13     0
14     9
15     9
16     7
17     0
18     8
19     0
20     4
21    11
22     5
23     4
24     1
25     1
26     7
27     0
28     2
29     2
30     5
31     5
Name: key, dtype: int64

In [44]:
featureset2 = pd.read_csv("Data/FeaturesLimited.csv")
featureset2 = featureset2.drop(columns=['Unnamed: 0'])

In [45]:
df=spark.createDataFrame(featureset2)
testdf = spark.createDataFrame(featureset)

In [46]:
RANDOM_SEED = 13579
transformed_df = df.rdd.map(lambda row: LabeledPoint(row[-1], Vectors.dense(row[0:-1])))
transformed_testdf = testdf.rdd.map(lambda row: LabeledPoint(row[-1], Vectors.dense(row[0:-1])))

# Training the Classifier
rf = RandomForest.trainClassifier(transformed_df, numClasses=7, categoricalFeaturesInfo={}, \
    numTrees=25, featureSubsetStrategy="auto", impurity="gini", \
    maxDepth=10, maxBins= 32, seed=RANDOM_SEED)

predictions = rf.predict(transformed_testdf.map(lambda x: x.features))
labels_and_predictions = transformed_testdf.map(lambda x: x.label).zip(predictions)
acc = labels_and_predictions.filter(lambda x: x[0] == x[1]).count() / float(transformed_testdf.count())
print("Model accuracy: %.3f%%" % (acc * 100))

Model accuracy: 59.375%


In [47]:
labels_and_predictions.take(135)

[(1.0, 3.0),
 (3.0, 2.0),
 (3.0, 3.0),
 (3.0, 2.0),
 (3.0, 2.0),
 (3.0, 4.0),
 (3.0, 4.0),
 (3.0, 3.0),
 (3.0, 4.0),
 (4.0, 4.0),
 (4.0, 4.0),
 (4.0, 4.0),
 (4.0, 4.0),
 (4.0, 5.0),
 (4.0, 5.0),
 (4.0, 4.0),
 (5.0, 5.0),
 (5.0, 6.0),
 (5.0, 5.0),
 (5.0, 5.0),
 (-1.0, 2.0),
 (5.0, 5.0),
 (5.0, 5.0),
 (5.0, 6.0),
 (5.0, 5.0),
 (5.0, 5.0),
 (-1.0, 1.0),
 (6.0, 6.0),
 (6.0, 6.0),
 (6.0, 6.0),
 (6.0, 6.0),
 (6.0, 6.0)]

In [50]:
index = 0
hitsongs = []
predictionlist = predictions.collect()
for x in predictionlist:
    if x < 3.0:
        hitsongs.append(index)
    index+=1

In [51]:
hitsongs

[1, 3, 4, 20, 26]

In [53]:
update = "One of these songs will rank in the Top 15 Songs of Next Week's Billboard Hot 100 Chart:\n\n"
sort = []
def takeSecond(elem):
    return elem[2]
for idx, values in newtracks.iterrows():
    if idx in hitsongs:
        sort.append([values['Song'], values['Performer'], values['spotify_track_popularity']])
sort = sorted(sort, key=takeSecond, reverse=True)
num = 0
for x in sort:
    if num < 5:
        update = update + x[0]+' by '+x[1]+'\n'
        num+=1
print(update)

One of these songs will rank in the Top 15 Songs of Next Week's Billboard Hot 100 Chart:

Sunday Best by Surfaces
Blueberry Faygo by Lil Mosey
WHATS POPPIN by Jack Harlow
Ride It by Regard
Level of Concern by Twenty One Pilots



In [54]:
auth = tweepy.OAuthHandler('TvrYJIPHns4OPJCetpYg2RJw4', 'juNWwVL0l527PMTMCni6NQaFB5e5Ksjf3DXBreSkGCjdTEjzYF')
auth.set_access_token('121839199-fJpZUrP3IS4R4MvoW8kaGg3PevusBjTSXu56mbe4', '5kAx7woytQWNRWuxuwgGAKX61onlluULLgPqHSXnfx6bW')

api = tweepy.API(auth)

In [55]:
api.update_status(update)

Status(_api=<tweepy.api.API object at 0x1a33de3710>, _json={'created_at': 'Thu Jun 04 23:03:30 +0000 2020', 'id': 1268679800719368200, 'id_str': '1268679800719368200', 'text': "One of these songs will rank in the Top 15 Songs of Next Week's Billboard Hot 100 Chart:\n\nSunday Best by Surfaces\nB… https://t.co/J4TIJQJNhX", 'truncated': True, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': [{'url': 'https://t.co/J4TIJQJNhX', 'expanded_url': 'https://twitter.com/i/web/status/1268679800719368200', 'display_url': 'twitter.com/i/web/status/1…', 'indices': [117, 140]}]}, 'source': '<a href="https://github.com/pkhiyara/Predicting-The-Next-Hit-Song" rel="nofollow">Twitter Hit Song Predictor</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 121839199, 'id_str': '121839199', 'name': 'PK', 'screen_name': '__PK2_', 'location': '', 'description':

In [ ]:
time.sleep(604800)